# Models

## What is a model?

- A “model” is a general specification of relationships among variables. 
    + e.g. a linear regression, such as: $ Price = \beta_1*Time +  \beta_0 (+ \epsilon)$
- A “trained model” is a particular model that has been built using some training data.
    + If the model is **parametric** (like a linear regression), then it has parameters that have been calculated using the training data;
    + If the model is **non-parametric**, then it has (not parameters but) an algorithm that has been constructed using the training data.
    
![Img](https://d1jnx9ba8s6j9r.cloudfront.net/blog/wp-content/uploads/2019/04/Parametric-vs-Non-Parametric-model-Artificial-Intelligence-Interview-Questions-Edureka.png)

## Linear Regression

For two variables $X$ and $Y$, each with $n$ values:

$\Large\sigma_{XY} = \frac{\Sigma^n_{i = 1}(x_i - \mu_x)(y_i - \mu_y)}{n}$ <br/>

`np.cov(X, Y, ddof=0)`

Pearson Correlation:<br/>$\Large r_P = \frac{\Sigma^n_{i = 1}(x_i - \mu_x)(y_i - \mu_y)}{\sqrt{\Sigma^n_{i = 1}(x_i - \mu_x)^2\Sigma^n_{i = 1}(y_i -\mu_y)^2}}$

Note that we are simply standardizing the covariance by the standard deviations of X and Y (the $n$'s cancel!).

`np.corrcoef(X, Y)`

**Where X and Y are lists of X and Y values.**

Similarly, you can use SciPy:<br>
`stats.pearsonr(X, Y)`

### Regression equations

The solution for a simple regression best-fit line is as follows:

- slope: <br/>$\Large m = r_P\frac{\sigma_y}{\sigma_x} = \frac{cov(X, Y)}{var(X)}$

- y-intercept:<br/> $\Large b = \mu_y - m\mu_x$

### Using Stats Model

`sm.formula.ols(formula="y ~ x", data=test_df).fit().summary()` <br>
Where your x:y data is in a dataframe.<br><br>
or with seperate x and y, you can use:<br>
`sm.OLS(y,x)` where y is a dataframe with y values and x is a dataframe with x values.

Use `sm.add_constant(df[x])` to add a constant column into a new dataframe for x when using `sm.OLS`

### $R^2$

$R^2$, the *coefficient of determination*, is a measure of how well the model fits the data.<br>
How well variation in x explains variation in y.

Interpreted as a percentage of change in y.

"$R^2$ change in Y is explained by change in X"

The actual calculation of $R^2$ is: <br/> $\Large R^2\equiv 1-\frac{\Sigma_i(y_i - \hat{y}_i)^2}{\Sigma_i(y_i - \bar{y})^2}$.

Adjusted $R^2$ adds a penalty for the complexity of the model. <br>
Adding more predictors increases the penalty dependent on the significance of the added predictors.

### Using `sklearn`

`from sklearn.linear_model import LinearRegression`

Initialize the regression by setting a variable = `LinearRegression()`

Then fit the model using:<br>
`variable.fit(df['x'], y)`

if x is one dimensional, you made need to use `lr.fit(X.reshape(-1, 1), y)`

For your $\beta_1$ or x coefficient, `variable.coef_`

For your $\beta_o$ or y-intercept, `variable.intercept_`

To access the model `variable.predict(X.reshape(-1, 1))`

**$R^2$**

To get your $R^2$ using `sklearn`, use<br>
```
from sklearn.metrics import r2_score

r2_score(y, model)
```

where model is your model from the previous section

**Other metrics**

To get other metrics about your model, use <br>
```
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("Metrics:")
# R2
print(f"R2: {r2_score(y, model):.3f}")
# MAE
print(f"Mean Absolute Error: {mean_absolute_error(y, model):.3f}")
# MSE
print(f"Mean Squared Error: {mean_squared_error(y, model):.3f}")
# RMSE - just MSE but set squared=False
print(f"Root Mean Squared Error: {mean_squared_error(y, model, squared=False):.3f}")
```


### Assumptions

#### Linearity

**The relationship between the target and predictor is linear.** Check this by drawing a scatter plot of your predictor and your target, and see if there is evidence that the relationship might not follow a straight line.

To test, run Pearson's r Coefficient and scatterplot

To test multicollinearity:

.corr(), heatmap or **Variance Inflation Factor (VIF)** using:<br>
```
from statsmodels.stats.outliers_influence import variance_inflation_factor

# defining an empty dataframe to capture the VIF scores
vif = pd.DataFrame()

# For each column,run a variance_inflaction_factor against all other columns to get a VIF Factor score
vif["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(len(X_train.columns))]

# label the scores with their related columns
vif["features"] = X_train.columns
```

$VIF < 10$ does not have high multicollinearity.

#### Independence

**The errors are independent**. In other words: Knowing the error for one point doesn't tell you anything about the error for another.

#### Normality

**The errors are normally distributed.** That is, smaller errors are more probable than larger errors, according to the familiar bell curve.

Use QQ plot with residuals. Residuals are calculated as:
$$ residual = y-actual - y-predicted $$

```
# QQ plots are generally great tools for checking for normality.
import statsmodels.api as sm

sm.qqplot(train_residuals, line = 'r');
```
We want the points to fit the plotted line.

A histogram of residuals should also be normally distributed around 0.

#### Homoskedasticity

**The errors are homoskedastic.** That is, the errors have the same variance. 

To test for homoskedasticity, scatter plot residuals.

Residuals should be pattern-less.

- **The ideal scenario**

    - Random scatter
    - Scattered around 0
    - No identifiable trend
    
    <img src="images/normal-resid.png" width="550">  
    
- **Non-linear relationship**

    - Clear non-linear scatter, but
    - Identifiable trend
    - **Fix:** Introduce polynomial terms
    - **Fix:** Variable transformation
    
    <img src="images/polynomial-resid.png" width="550">

- **Autocorrelation**

    - Identifiable trend, or
    - Consecutively positive/negative residuals
    - **Fix:** Consider sequential analysis methods (which we'll discuss in phase 4)
    
    <img src="images/autocorrelation.png" width="550">

- **Heteroskedasticity**

    - The spread of residuals is different at different levels of the fitted values
    - **Fix:** Variable transformation (log)  
    
    <img src="images/heteroskedasticity.png" width="550">
    


For interpretation after **log transformation**, interpret coef as:

Change of 1 in $x$ is a $(e^{coef} -1) \cdot 100$ % change in $y$ **holding all else constant**.

## Multiple Linear Regressions

Can add categorical vars to regression.

We are also adding predictive variables, so our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

**Closed-form solution (matrices)**<br>
In a word, for a multiple linear regression problem where $X$ is the matrix of independent variable values and $y$ is the vector of dependent variable values, the vector of optimizing regression coefficients $\vec{b}$ is given by:

$\vec{b} = (X^TX)^{-1}X^Ty$.

**Dummying categorical data**

Avoid dummying numeric variables or variables of high cardinality.

You can bin variables of high cardinality to reduce the number of coefficients created by dummying that variable.

Should result in a table of 0s and 1s for categorical data.

Use `pd.get_dummies()` to create dummy columns for a given df

or use One Hot as per example:
```
# Let's try using sklearn's OneHotEncoder to create our dummy columns:

ohe = OneHotEncoder(drop='first')
comma_trans = ohe.fit_transform(comma_use.drop('RespondentID', axis=1))
```

### Finding your correlated $x$'s

When choosing features, you can use a correlation table:<br>
`df.corr()`

and plot a heat map using seaborn<br>
```
sns.set(rc={'figure.figsize':(8, 8)})

# Use the .heatmap method to depict the relationships visually!
sns.heatmap(wine.corr());
```

but the best way to find the predictors with the highest correlations you can use this:
```
df_corrs = df.corr()['y'].map(abs).sort_values(ascending=False)
df_corrs
```

### Recursive feature elimination

Removes lowest correlating predictors in data set up to a given number of features remaining.

```
select = RFE(lr_rfe, n_features_to_select=3)

# Fit standard scaled model on X and y-variable
select.fit(X=df_scaled, y=df['y'])

# Run ranking on your columns.
select.ranking_

```

> **Caution**: RFE is probably not a good strategy if your initial dataset has many predictors. It will likely be easier to start with a *simple* model and then slowly increase its complexity. This is also good advice for when you're first getting your feet wet with `sklearn`!



### Standard Scale

Benefits:

- This tends to make values relatively small (mean value is at $0$ and one standard deviation $\sigma$ from the mean is $1$).
- **Easier interpretation: larger coefficients tend to be more influential**

Use the following method to standardize the scale of your data:
```
df_scaled = (df - np.mean(df)) / np.std(df)
```
Essentially, you're taking a z-score for every value to standardize the coefs.

Thus, the coefs can be interpreted as for every $\sigma$ increase in $x$, $y$ changes by coef.

With coefs standardized, you can evaluate and remove predictors of low magnitude contributing predictors to reduce model complexity.

To standard scale in `sklearn` use 
```
ss = StandardScaler()
ss.fit(df)
data_std_scale = ss.transform(df)
```
Once the data is standardized, you can fit it with a linear regression:
```
lr = LinearRegression()
lr.fit(data_std_scale, test_df)
```

And get your metrics using the following code:
```
print("Metrics:")
# Intercept
print(f"y-intercept: {lr.intercept_:.3f}")
# Coefs
print(f"x coefficients: {lr.coef_:.3f}")
# R-Squared
print(f"R2: {lr.score(data_std_scale, test_df):.3f}")
```



## Predictive Models

Predictive models are more concerned with the output of a model ($y$) rather than coefs or intercepts.

Because of this, we're not concerned with transformations or number of predictors. More predictors is probably better.

We can also forego some of our linear regression assumptions that we must hold for inferential statistics.

Primary concern is reducing error.

Thus, we want to use the following to evaluate our model:
- Root Mean Error
- Mean Absolute Error
- Mean Squared Error

$\Large Total\ Error\ = Prediction\ Error+ Irreducible\ Error$

Our prediction error can be further broken down into error due to bias and error due to variance.

$\Large Total\ Error = Model\ Bias^2 + Model\ Variance + Irreducible\ Error$

### Variance vs Bias